Naive RAG + ChatGPT

In [2]:
! pip install openai
! pip install python-dotenv
! pip install pymupdf4llm
! pip install matplotlib
! pip install chromadb
! pip install llama-index
! pip install llama-index-vector-stores-chroma

from openai import OpenAI
import os 
from dotenv import load_dotenv
import pymupdf4llm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# Lezen document
# documents = pymupdf4llm.to_markdown("Leidraad kwaliteit AI in de zorg NL.pdf")
md_text = open("leidraad.txt")

# Chunking regels
chunk_size = 1000  # Size of each chunk
overlap = 200      # Number of overlapping characters

# API key en OpenAI laden
load_dotenv()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

Laden document

In [3]:
# --- Step 1: Load Document ---
# Read the plain text file
with open("leidraad.txt", "r", encoding="ISO-8859-1") as file:
    document = file.read()  # Read content into a string

ChromaDB aanmaken voor creeren vector database

In [4]:
# --- Step 2: Store Text in ChromaDB ---
# Initialize ChromaDB
chroma_client = chromadb.Client()
collection = chroma_client.create_collection("leidraad_ai_in_zorg")

Split in chunks

In [5]:

chunks = [
    document[i:i+chunk_size] 
    for i in range(0, len(document) - chunk_size + 1, chunk_size - overlap)
]

# Add chunks to ChromaDB
for idx, chunk in enumerate(chunks):
    collection.add(
        ids=[str(idx)],
        documents=[chunk]
    )

# Uncomment to print chunks for verification
# for idx, chunk in enumerate(chunks):
#     print(f"Chunk {idx}:\n{chunk}\n{'-'*80}")


User query

In [6]:
user_query = "Wat is de uitdrukking van deze leidraad?"

Relevante chunks ophalen

In [7]:
# Retrieve relevant chunks using Chroma
results = collection.query(
    query_texts=[user_query],
    n_results=10
)
retrieved_text = results['documents'][0][0]  # Top relevant chunk
print(retrieved_text)

esteld met eisen en criteria voor ontwikkeling,

validatie, evaluatie en implementatie van een AIPA in de zorg.

Deze leidraad is tot stand gekomen door de inzet van vele betrokkenen. In het bijzonder

danken wij Rosalie van Oostrom en Pieter Boone van VWS voor het organiseren en

coördineren van de totstandkoming van deze leidraad, Roy Tomeij voor het voorzitten van de

werkgroepen en Rachel Peeters voor de ondersteuning tijdens en na de werkgroepen. Wij

danken alle actieteamleden en de werkgroepleden voor hun actieve deelname, inzet en

betrokkenheid in de werkgroepen die hebben geleid tot deze leidraad. Daarnaast danken wij

alle reviewers en de deelnemers aan de praktijktoetsen voor de vele feedbackcommentaren

die hebben geleid tot een verbetering van de conceptversies, en KPMG voor het coördineren

en organiseren van dit proces in samenwerking met de redactieraad. Verder danken we de

NEN Nederlandse AI Medical Device expert Group, Patiëntenfederatie Nederland en

Inspectie Jeug

Vraag antwoord op basis van top chunks

In [8]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": f"Je bent een assistent die vragen over de Leidraad AI in de zorg beantwoordt."},
        {"role": "user", "content": f"Baseer je antwoord op deze tekst: {retrieved_text}\n\n{user_query}"}
    ]
)

# --- Step 5: Output Result ---
print("Vraag:", user_query)
print("Antwoord:", response.choices[0].message.content)

Vraag: Wat is de uitdrukking van deze leidraad?
Antwoord: Het lijkt erop dat je de doelstelling of het doel van de genoemde leidraad wilt weten. Hoewel de door jou gegeven tekst deze informatie niet expliciet biedt, verwijst hij wel naar de ontwikkeling, validatie, evaluatie en implementatie van AI in de zorg. Daaruit zou je kunnen opmaken dat het doel van de leidraad is om een reeks richtlijnen en criteria te bieden om de kwaliteit en effectiviteit van AI in de zorg te waarborgen. De leidraad is tot stand gekomen door de bijdragen en feedback van vele betrokkenen, wat suggereert dat deze bedoeld is om zo breed mogelijk toepasbaar te zijn in de zorgsector.
